# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [64]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import geoviews as gv
import holoviews as hv
from holoviews import opts
#import geoviews.tile_sources as gts
import geoviews.tile_sources as gts
from holoviews import opts
# Import API key
from api_keys2 import geoapify_key

In [55]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ponta delgada,37.7333,-25.6667,297.55,88,40,3.09,PT,1722894578
1,1,yellowknife,62.4560,-114.3525,295.60,41,75,1.79,CA,1722894579
2,2,ratnagiri,16.9833,73.3000,298.97,90,100,4.32,IN,1722894580
3,3,feodosiya,45.0333,35.3833,297.28,62,49,5.13,UA,1722894582
4,4,coahuayana de hidalgo,18.7000,-103.6583,304.22,67,97,3.94,MX,1722894583
...,...,...,...,...,...,...,...,...,...,...
567,567,ebaye,8.7748,167.7392,301.16,83,0,3.09,MH,1722895216
568,568,touros,-5.1989,-35.4608,297.23,77,46,6.77,BR,1722894957
569,569,nova vicosa,-17.8919,-39.3719,295.19,75,0,3.04,BR,1722895218
570,570,kambarka,56.2694,54.2062,288.42,94,0,1.83,RU,1722895219


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [67]:
#%%capture --no-display

points = gv.Points(city_data_df, kdims=['Lng', 'Lat'], vdims=['City', 'Country'])

# Customize the plot
points = points.opts(
    opts.Points(
        size=8, color='blue', tools=['hover'], width=800, height=600, 
        title="Cities and Hotels Worldwide"
    )
)

# Display the map with OpenStreetMap as the base
map = gts.OSM * points

# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [58]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df.loc[
    (city_data_df['Lat'] > 21)& 
    (city_data_df['Lat'] < 35)& 
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,portales,34.1862,-103.3344,307.70,14,0,4.29,US,1722894641
111,111,galveston,29.3669,-94.9669,308.72,63,0,4.12,US,1722894701
135,135,cabo san lucas,22.8909,-109.9124,306.76,53,0,4.12,MX,1722894558
196,196,uruzgan,32.8333,66.0000,298.68,38,0,0.63,AF,1722894800
254,254,kerman,30.2832,57.0788,298.07,22,0,2.06,IR,1722894863
392,392,timimoun,29.2639,0.2310,313.36,10,0,2.06,DZ,1722895018
443,443,qa'en,33.7265,59.1844,297.27,25,0,3.61,IR,1722895078


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head

<bound method NDFrame.head of                City Country      Lat       Lng  Humidity Hotel Name
58         portales      US  34.1862 -103.3344        14           
111       galveston      US  29.3669  -94.9669        63           
135  cabo san lucas      MX  22.8909 -109.9124        53           
196         uruzgan      AF  32.8333   66.0000        38           
254          kerman      IR  30.2832   57.0788        22           
392        timimoun      DZ  29.2639    0.2310        10           
443           qa'en      IR  33.7265   59.1844        25           >

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 100000

params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params= params)
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
portales - nearest hotel: Best Western Plus
galveston - nearest hotel: Comfort Inn & Suites Texas City
cabo san lucas - nearest hotel: Comfort Rooms
uruzgan - nearest hotel: No hotel found
kerman - nearest hotel: حافظ
timimoun - nearest hotel: أناف
qa'en - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
58,portales,US,34.1862,-103.3344,14,Best Western Plus
111,galveston,US,29.3669,-94.9669,63,Comfort Inn & Suites Texas City
135,cabo san lucas,MX,22.8909,-109.9124,53,Comfort Rooms
196,uruzgan,AF,32.8333,66.0000,38,No hotel found
254,kerman,IR,30.2832,57.0788,22,حافظ
392,timimoun,DZ,29.2639,0.2310,10,أناف
443,qa'en,IR,33.7265,59.1844,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [65]:
points = gv.Points(hotel_df, kdims=['Lng', 'Lat'], vdims=['City', 'Country', 'Hotel Name'])


points = points.opts(
    opts.Points(
        size=8, color='blue', tools=['hover'], width=800, height=600, 
        title="Cities and Hotels Worldwide"
    )
)

map = gts.OSM * points


map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)